In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder 
import pickle


In [15]:
#load trained model and all pickle files
model=load_model('model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder_geography.pkl', 'rb') as f:
    one_hot_encoder_geography = pickle.load(f)

In [16]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [19]:

geography_encoded = one_hot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geography_encoded_df = pd.DataFrame(geography_encoded, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geography_encoded_df

c:\Users\ACER\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_data = pd.DataFrame([input_data])
input_data = pd.concat([input_data.reset_index(drop=True), geography_encoded_df], axis=1)

In [23]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [24]:
input_data['Gender']=label_encoder_gender.transform([input_data['Gender']])
input_data

c:\Users\ACER\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [25]:
input_data=input_data.drop(['Geography'], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
scaled_input = scaler.transform(input_data)
scaled_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.41421356,
         0.        , -1.41421356]])

In [30]:
prediction = model.predict(scaled_input)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[0.19719632]], dtype=float32)

In [31]:
prediction_probability = prediction[0][0]
prediction_probability

0.19719632

In [32]:
if prediction_probability > 0.5:
    print(f'The customer is likely to leave the bank with a probability of {prediction_probability:.2f}')
else:
    print(f'The customer is likely to stay with the bank with a probability of {1-prediction_probability:.2f}')

The customer is likely to stay with the bank with a probability of 0.80
